In [7]:
import numpy as np
import pandas as pd

def calculate_element_local_stiffness_matrix(E, A, L):
    ke_local = np.zeros((4, 4), dtype=float)
    ke_local[0, 0] = E * A / L
    ke_local[0, 2] = -E * A / L
    ke_local[2, 0] = -E * A / L
    ke_local[2, 2] = E * A / L
    return ke_local

def define_element_rotation_matrix(theta):
    beta = np.zeros((4, 4), dtype=float)
    unit_rotation = np.array([[np.cos(theta), np.sin(theta)],
                               [-np.sin(theta), np.cos(theta)]])
    beta[:2, :2] = unit_rotation
    beta[2:, 2:] = unit_rotation
    return beta

def calculate_element_length(node_i_coordinates, node_j_coordinates):
    return np.linalg.norm(np.array(node_j_coordinates) - np.array(node_i_coordinates))

def calculate_element_orientation(node_i_coordinates, node_j_coordinates):
    delta = np.array(node_j_coordinates) - np.array(node_i_coordinates)
    return np.arctan2(delta[1], delta[0])

def define_global_element_stiffness_matrix(ke_local, beta):
    return beta.T @ ke_local @ beta

def define_global_dof_for_element(node_number_i, node_number_j):
    dof_i = [node_number_i * 2, node_number_i * 2 + 1]
    dof_j = [node_number_j * 2, node_number_j * 2 + 1]
    return dof_i + dof_j

def assemble_global_stiffness_matrix(num_dof, ke_global, element_global_dof):
    K = np.zeros((num_dof, num_dof), dtype=float)
    for i in range(len(ke_global)):
        dofs = element_global_dof[i]
        K[np.ix_(dofs, dofs)] += ke_global[i]
    return K



In [29]:

node_data = pd.read_csv("Nodes.csv")
element_data = pd.read_csv("Elements.csv")

# Define nodes
nodes = [[node_data.X[i], node_data.Y[i]] for i in range(len(node_data))]

# Define material properties (E)
material_properties = [29500000.0]

# Define section properties (A)
section_properties = [0.877, 0.30]

# Define elements
elements = [[element_data.node_i[i] - 1, element_data.node_j[i] - 1, 
             element_data.mat_prop[i] - 1, element_data.sect_prop[i] - 1] 
            for i in range(len(element_data))]

# Fixed degrees of freedom 
s = np.array([0, 1, 23]) 

# Displacement vector
num_dof = len(nodes) * 2
u = np.zeros(num_dof)

# Element properties
L = [calculate_element_length(nodes[e[0]], nodes[e[1]]) for e in elements]
θ = [calculate_element_orientation(nodes[e[0]], nodes[e[1]]) for e in elements]
A = [section_properties[e[3]] for e in elements]
E = [material_properties[e[2]] for e in elements]

# Element stiffness matrices
ke_local = [calculate_element_local_stiffness_matrix(E[i], A[i], L[i]) for i in range(len(elements))]
β = [define_element_rotation_matrix(θ[i]) for i in range(len(elements))]
ke_global = [define_global_element_stiffness_matrix(ke_local[i], β[i]) for i in range(len(elements))]
element_global_dof = [define_global_dof_for_element(elements[i][0], elements[i][1]) for i in range(len(elements))]

# Big Mac
K = assemble_global_stiffness_matrix(num_dof, ke_global, element_global_dof)

# External force vector
F = np.zeros(num_dof)
w = 230.67 / 12  # lbs/ft converted to lbs/in, 230.67 from hw 2

# Apply uplift uniform load on top chord elements
top_chord_elements = range(11)

for i in top_chord_elements:
    F[2 * elements[i][0] + 1] += w * L[i] / 2
    F[2 * elements[i][1] + 1] += w * L[i] / 2

# Partitioning for free and fixed DOFs
p = np.setdiff1d(np.arange(num_dof), s)  # Free DOFs

# Extract submatrices
Kpp = K[np.ix_(p, p)]
Kss = K[np.ix_(s, s)]
Kps = K[np.ix_(p, s)]
Ksp = K[np.ix_(s, p)]

Fp = F[p]

# Solve for displacements
us = u[s]  # Imposed displacements (if any)
up = np.linalg.solve(Kpp, Fp - Kps @ us)

# Fill displacement vector
u[p] = up

# Find maximum uplift deflection
max_deflection = np.max(u[1::2])  #only Y displacements
max_deflection_index = np.argmax(u[1::2])

# Solve for reactions
Fs = Ksp @ up + Kss @ us

# Solve for element internal forces
u_element_global = [u[dof] for dof in element_global_dof]
δ = [β[i] @ u_element_global[i] for i in range(len(elements))]
P = [ke_local[i] @ δ[i] for i in range(len(elements))]

print(F)
print(f"Maximum uplift deflection: {max_deflection:.5f} inches at node {max_deflection_index + 1}")
print(f"Reaction forces: {Fs}")
print(f"Internal forces per element: {P}")
print(f"Internal forces per element 9: {P[8]}")

[   0.     576.675    0.    1038.015    0.     922.68     0.     922.68
    0.     922.68     0.     922.68     0.     922.68     0.     922.68
    0.     922.68     0.     922.68     0.    1153.35     0.     692.01
    0.       0.       0.       0.       0.       0.       0.       0.
    0.       0.       0.       0.       0.       0.       0.       0.
    0.       0.       0.       0.       0.       0.   ]
Maximum uplift deflection: 1.98345 inches at node 18
Reaction forces: [ 2.03726813e-10 -4.84407000e+03 -4.72873500e+03]
Internal forces per element: [array([-4327.12303478,     0.        ,  4327.12303478,     0.        ]), array([-10694.17550024,      0.        ,  10694.17550024,      0.        ]), array([-15639.45896856,      0.        ,  15639.45896856,      0.        ]), array([-19266.00017866,      0.        ,  19266.00017866,      0.        ]), array([-21573.79913054,      0.        ,  21573.79913054,      0.        ]), array([-22562.8558242,      0.       ,  22562.8558242,   